# Task
Implement a command-line password vault tool in Python with features for adding, retrieving, and deleting password entries, using Fernet encryption and JSON file storage, and include basic testing and documentation.

## Setup and dependencies

### Subtask:
Install any necessary libraries (e.g., `cryptography` for Fernet encryption).


**Reasoning**:
The subtask is to install the `cryptography` library. This can be done using the pip package manager within a code block.



In [1]:
%pip install cryptography

## Encryption/decryption module

### Subtask:
Create functions for encrypting and decrypting passwords using Fernet.


**Reasoning**:
Implement the functions for generating a key, encrypting, and decrypting data using Fernet as per the instructions.



In [2]:
from cryptography.fernet import Fernet

def generate_key():
  """Generates a new Fernet key."""
  return Fernet.generate_key()

def encrypt_data(key, data):
  """Encrypts data using a Fernet key."""
  try:
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode())
    return encrypted_data
  except Exception as e:
    print(f"Encryption error: {e}")
    return None

def decrypt_data(key, encrypted_data):
  """Decrypts data using a Fernet key."""
  try:
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode()
    return decrypted_data
  except Exception as e:
    print(f"Decryption error: {e}")
    return None

## Password vault class/functions

### Subtask:
Implement functions to add, retrieve, and delete password entries. Use a dictionary to store the password entries in memory.


**Reasoning**:
Implement the PasswordVault class with methods for adding, retrieving, and deleting entries as specified in the instructions.



In [3]:
class PasswordVault:
  """A simple in-memory password vault."""

  def __init__(self):
    """Initializes an empty password vault."""
    self.entries = {}

  def add_entry(self, service, username, password):
    """Adds a new password entry."""
    self.entries[service] = {"username": username, "password": password}
    print(f"Entry for {service} added.")

  def retrieve_entry(self, service):
    """Retrieves a password entry."""
    return self.entries.get(service)

  def delete_entry(self, service):
    """Deletes a password entry."""
    if service in self.entries:
      del self.entries[service]
      print(f"Entry for {service} deleted.")
    else:
      print(f"Entry for {service} not found.")


## File handling

### Subtask:
Implement functions to save the password vault to a file (e.g., JSON) after encryption. Implement functions to load the password vault from the file, decrypting it upon loading.


**Reasoning**:
Implement the `save_vault` and `load_vault` functions with error handling for file operations and encryption/decryption, using the previously defined encryption and decryption functions and the `PasswordVault` class.



In [5]:
import json

def save_vault(vault, file_path, key):
  """Saves the password vault to a file after encryption."""
  try:
    vault_data = json.dumps(vault.entries)
    encrypted_data = encrypt_data(key, vault_data)
    if encrypted_data:
      with open(file_path, 'wb') as f:
        f.write(encrypted_data)
      print(f"Vault saved to {file_path}")
    else:
      print("Failed to encrypt vault data.")
  except IOError as e:
    print(f"File saving error: {e}")
  except Exception as e:
    print(f"An unexpected error occurred during saving: {e}")

def load_vault(file_path, key):
  """Loads the password vault from a file and decrypts it."""
  try:
    with open(file_path, 'rb') as f:
      encrypted_data = f.read()

    decrypted_data = decrypt_data(key, encrypted_data)
    if decrypted_data:
      vault_data = json.loads(decrypted_data)
      vault = PasswordVault()
      vault.entries = vault_data
      print(f"Vault loaded from {file_path}")
      return vault
    else:
      print("Failed to decrypt vault data.")
      return None
  except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    return None
  except IOError as e:
    print(f"File loading error: {e}")
    return None
  except json.JSONDecodeError:
    print("Error: Could not decode vault data as JSON.")
    return None
  except Exception as e:
    print(f"An unexpected error occurred during loading: {e}")
    return None

In [6]:
import os

KEY_FILE = 'vault.key'
VAULT_FILE = 'vault.json'

def main():
    """Handles the CLI interactions for the password vault."""
    key = None
    if os.path.exists(KEY_FILE):
        with open(KEY_FILE, 'rb') as f:
            key = f.read()
        print("Loaded existing encryption key.")
    else:
        key = generate_key()
        with open(KEY_FILE, 'wb') as f:
            f.write(key)
        print("Generated and saved new encryption key.")

    vault = None
    if os.path.exists(VAULT_FILE):
        vault = load_vault(VAULT_FILE, key)
        if vault is None:
            print("Failed to load vault. Starting with an empty vault.")
            vault = PasswordVault()
    else:
        print("Vault file not found. Starting with an empty vault.")
        vault = PasswordVault()

    while True:
        command = input("Enter command (add, retrieve, delete, save, load, quit): ").strip().lower()

        if command == 'add':
            service = input("Enter service name: ").strip()
            if not service:
                print("Service name cannot be empty.")
                continue
            username = input("Enter username: ").strip()
            password = input("Enter password: ").strip()
            vault.add_entry(service, username, password)

        elif command == 'retrieve':
            service = input("Enter service name: ").strip()
            if not service:
                print("Service name cannot be empty.")
                continue
            entry = vault.retrieve_entry(service)
            if entry:
                print(f"Service: {service}, Username: {entry['username']}, Password: {decrypt_data(key, entry['password'])}")
            else:
                print(f"Entry for {service} not found.")

        elif command == 'delete':
            service = input("Enter service name: ").strip()
            if not service:
                print("Service name cannot be empty.")
                continue
            vault.delete_entry(service)

        elif command == 'save':
            save_vault(vault, VAULT_FILE, key)

        elif command == 'load':
            loaded_vault = load_vault(VAULT_FILE, key)
            if loaded_vault:
                vault = loaded_vault

        elif command == 'quit':
            break

        else:
            print("Invalid command. Please try again.")

if __name__ == "__main__":
    main()

Generated and saved new encryption key.
Vault file not found. Starting with an empty vault.
Enter command (add, retrieve, delete, save, load, quit): add
Enter service name: communicate
Enter username: garvey
Enter password: 12345678
Entry for communicate added.
Enter command (add, retrieve, delete, save, load, quit): quit
